In [1]:
# packages
import pandas as pd
from bs4 import BeautifulSoup
import requests
from zipfile import ZipFile
import requests
import re
from datetime import date
import json
import xmltodict
import xml.etree.ElementTree as ET
import numpy as np
import os

In [2]:
# get links from website

URL = "https://ds.marktstammdatenregister.dev/Marktstammdatenregister"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
DataLink = soup.findAll('a', href = re.compile('^https://download.marktstammdatenregister.de/Gesamtdatenexport_'))[0]['href']

!!! Download takes ~15 minutes !!!

In [3]:
# download data
req = requests.get(DataLink)

In [4]:
# set filename
datum = date.today()
filename = f'MaStR_Gesamtdatenexport_{datum}.zip'


In [5]:
# writing file to system
print('Download started')
with open(filename, 'wb') as output_file:
    output_file.write(req.content)
print('Download completed')

Download started
Download completed


In [6]:
zip_path = "/home/jan/Uni/DS-Project/modules/data/" + filename
with ZipFile(zip_path, 'r') as ZipObject:
    print(ZipObject.printdir())

File Name                                             Modified             Size
AnlagenEegBiomasse.xml                         2022-12-12 01:00:22     29738234
AnlagenEegGeoSolarthermieGrubenKlaerschlammDruckentspannung.xml 2022-12-12 01:00:22       268836
AnlagenEegSolar_1.xml                          2022-12-12 01:00:36    165866028
AnlagenEegSolar_10.xml                         2022-12-12 01:02:16    163695320
AnlagenEegSolar_11.xml                         2022-12-12 01:02:28    163065222
AnlagenEegSolar_12.xml                         2022-12-12 01:02:40    163683398
AnlagenEegSolar_13.xml                         2022-12-12 01:02:50    157221834
AnlagenEegSolar_14.xml                         2022-12-12 01:03:00    149668190
AnlagenEegSolar_15.xml                         2022-12-12 01:03:10    150006216
AnlagenEegSolar_16.xml                         2022-12-12 01:03:22    150044602
AnlagenEegSolar_17.xml                         2022-12-12 01:03:34    150473430
AnlagenEegSolar_18.xml 

In [7]:
mastr_directory = "/home/jan/Uni/DS-Project/data/MaStR/"
zip_path = "/home/jan/Uni/DS-Project/modules/data/" + filename
# unzip folder and extract required data
Matches = []
with ZipFile(zip_path, 'r') as ZipObject:
    for names in ZipObject.namelist():
        PVA = re.findall(r'^EinheitenSolar.*xml$', names)
        #Matches.append(PVA)
        WKA = re.findall(r'^EinheitenWind.*xml$', names)
        #Matches.append(WKA)
        NAP = re.findall(r'^Netzanschlusspunkte.*xml$', names)
        Matches.append(NAP)
        Netze = re.findall(r'^Netze.*xml$', names)
        Matches.append(Netze)
    while [] in Matches :
        Matches.remove([])
    Matches = [str(M) for M in Matches]
    for i in range(len(Matches)-1):
        Source_Name = Matches[i][2:-2]
        ZipObject.extract(
            Source_Name, path = mastr_directory
        )
# delete zip folder
#os.remove(zip_path)

In [8]:
def parse_XML(xml_file, df_cols): 
    """
    shamelessly stolen from:
    https://medium.com/@robertopreste/from-xml-to-pandas-dataframes-9292980b1c1c
    """
    
    xtree = ET.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0]))
        for el in df_cols[1:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

In [9]:
WKA_cols = ["EinheitMastrNummer",
           "EinheitMastrNummer",
           "AnlagenbetreiberMastrNummer",
           "Land",
           "Bundesland",
           "Landkreis",
           "Gemeinde",
           "Gemeindeschluessel",
           "Postleitzahl",
           "Inbetriebnahmedatum",
           "EinheitSystemstatus",
           "Nettonennleistung",
           # WKA specific
           "Laengengrad",
           "Breitengrad",
           "Einspeisungsart",
           "Hersteller",
           "Nabenhoehe",
           "Rotordurchmesser",
           "AuflagenAbschaltungSchallimmissionsschutzNachts",
           "AuflagenAbschaltungSchallimmissionsschutzTagsueber",
           "AuflagenAbschaltungSchattenwurf",
           "AuflagenAbschaltungTierschutz",
           "AuflagenAbschaltungEiswurf"
           ]
PVA_cols = ["EinheitMastrNummer",
            "AnlagenbetreiberMastrNummer",
            "Land",
            "Bundesland",
            "Landkreis",
            "Gemeinde",
            "Gemeindeschluessel",
            "Postleitzahl",
            "Inbetriebnahmedatum",
            "EinheitSystemstatus",
            "Nettonennleistung",
            "Einspeisungsart",
            "ZugeordneteWirkleistungWechselrichter",
            "AnzahlModule",
            "Lage",
            "Leistungsbegrenzung",
            "EinheitlicheAusrichtungUndNeigungswinkel",
            "Hauptausrichtung",
            "HauptausrichtungNeigungswinkel",
            "Nutzungsbereich"
            ]
NAP_cols = ["NetzanschlusspunktMastrNummer",
            "NetzanschlusspunktBezeichnung",
            "Lokationtyp",
            "Spannungsebene",
            "Nettoengpassleistung",
            "BilanzierungsgebietNetzanschlusspunktId",
            "NetzMaStRNummer",
            "NochInPlanung"]


In [11]:
# parse Wind file
WKA_df = parse_XML("/home/jan/Uni/DS-Project/data/MaStR/EinheitenWind.xml",
          WKA_cols)
os.remove("/home/jan/Uni/DS-Project/data/MaStR/EinheitenWind.xml")

!!! takes ~6 minutes to run !!!

In [5]:
Matches = []
with ZipFile("/home/jan/Uni/DS-Project/modules/data/MaStR_Gesamtdatenexport_2022-11-06", 'r') as ZipObject:
    for names in ZipObject.namelist():
        PVA = re.findall(r'^EinheitenSolar.*xml$', names)
        Matches.append(PVA)
        WKA = re.findall(r'^EinheitenWind.*xml$', names)
        Matches.append(WKA)
    while [] in Matches :
        Matches.remove([])
    Matches = [str(M) for M in Matches]

In [ ]:
# parse Solar files
PVA_df = pd.DataFrame()
# get PVA filenames
mastr_directory = "/home/jan/Uni/DS-Project/data/MaStR/"
for i in range(len(Matches)-1):
    Source_Name = Matches[i][2:-2]
    Unit = mastr_directory + Source_Name
    print(Source_Name)
    # fill PVA_df
    PVA_part = parse_XML(Unit, PVA_cols)
    PVA_df = PVA_df.append(PVA_part)
    os.remove(Unit)

In [10]:
Matches

["['Netzanschlusspunkte_1.xml']",
 "['Netzanschlusspunkte_10.xml']",
 "['Netzanschlusspunkte_11.xml']",
 "['Netzanschlusspunkte_12.xml']",
 "['Netzanschlusspunkte_13.xml']",
 "['Netzanschlusspunkte_14.xml']",
 "['Netzanschlusspunkte_15.xml']",
 "['Netzanschlusspunkte_16.xml']",
 "['Netzanschlusspunkte_17.xml']",
 "['Netzanschlusspunkte_18.xml']",
 "['Netzanschlusspunkte_19.xml']",
 "['Netzanschlusspunkte_2.xml']",
 "['Netzanschlusspunkte_20.xml']",
 "['Netzanschlusspunkte_21.xml']",
 "['Netzanschlusspunkte_22.xml']",
 "['Netzanschlusspunkte_23.xml']",
 "['Netzanschlusspunkte_24.xml']",
 "['Netzanschlusspunkte_3.xml']",
 "['Netzanschlusspunkte_4.xml']",
 "['Netzanschlusspunkte_5.xml']",
 "['Netzanschlusspunkte_6.xml']",
 "['Netzanschlusspunkte_7.xml']",
 "['Netzanschlusspunkte_8.xml']",
 "['Netzanschlusspunkte_9.xml']",
 "['Netze.xml']"]

In [ ]:
# parse Netzanschlusspunkte files
NAP_df = pd.DataFrame()
# get NAP filenames
mastr_directory = "/home/jan/Uni/DS-Project/data/MaStR/"
for i in range(len(Matches)-1):
    Source_Name = Matches[i][2:-2]
    Unit = mastr_directory + Source_Name
    print(Source_Name)
    # fill PVA_df
    NAP_part = parse_XML(Unit, NAP_cols)
    NAP_df = NAP_df.append(NAP_part)
    os.remove(Unit)

In [12]:
# write csv files to disc

print('Writing started')
#WKA_df.to_csv(mastr_directory + "WKA.csv")
#PVA_df.to_csv(mastr_directory + "PVA.csv")
NAP_df.to_csv(mastr_directory + "NAP.csv")
print('Writing completed')

Writing started
Writing completed
